In [1]:
import QuantLib as ql
import pandas as pd

In [2]:
def print_curve(xlist, ylist, precision=3):
    """
    Method to print curve in a nice format
    """
    print ("----------------------")
    print ("Maturities\tCurve")
    print ("----------------------")
    for x,y in zip(xlist, ylist):
        print (x,"\t\t", round(y, precision))
    print ("----------------------")
    

In [3]:
# Exhibit 5-5 given in Frank Fabozzi's Bond Markets, Analysis and Strategies, Sixth Edition.
# Deposit rates
depo_maturities = [ql.Period(6,ql.Months), ql.Period(12, ql.Months)]
depo_rates = [5.25, 5.5]

# Bond rates
bond_maturities = [ql.Period(6*i, ql.Months) for i in range(3,21)]
bond_rates = [5.75, 6.0, 6.25, 6.5, 6.75, 6.80, 7.00, 7.1, 7.15,
              7.2, 7.3, 7.35, 7.4, 7.5, 7.6, 7.6, 7.7, 7.8]

print_curve(depo_maturities+bond_maturities, depo_rates+bond_rates)



----------------------
Maturities	Curve
----------------------
6M 		 5.25
1Y 		 5.5
1Y6M 		 5.75
2Y 		 6.0
2Y6M 		 6.25
3Y 		 6.5
3Y6M 		 6.75
4Y 		 6.8
4Y6M 		 7.0
5Y 		 7.1
5Y6M 		 7.15
6Y 		 7.2
6Y6M 		 7.3
7Y 		 7.35
7Y6M 		 7.4
8Y 		 7.5
8Y6M 		 7.6
9Y 		 7.6
9Y6M 		 7.7
10Y 		 7.8
----------------------


### Build spot

In [60]:
calc_date = ql.Date(15, 1, 2015)
ql.Settings.instance().evaluationDate = calc_date

day_count = ql.Thirty360()
calendar = ql.UnitedStates()

bussiness_convention = ql.Unadjusted
end_of_month = False
settlement_days = 0
face_amount = 100
coupon_frequency = ql.Period(ql.Semiannual)
settlement_days = 0

# create deposit rate helpers from depo_rates
depo_helpers = [ql.DepositRateHelper(ql.QuoteHandle(ql.SimpleQuote(r/100.0)),
                                     m,
                                     settlement_days,
                                     calendar,
                                     bussiness_convention,
                                     end_of_month,
                                     day_count)
                for r, m in zip(depo_rates, depo_maturities)]

# create fixed rate bond helpers from fixed rate bonds
bond_helpers = []
for r, m in zip(bond_rates, bond_maturities):
    termination_date = calc_date + m
    schedule = ql.Schedule(calc_date,
                   termination_date,
                   coupon_frequency,
                   calendar,
                   bussiness_convention,
                   bussiness_convention,
                   ql.DateGeneration.Backward,
                   end_of_month)

    helper = ql.FixedRateBondHelper(ql.QuoteHandle(ql.SimpleQuote(face_amount)), # price=face value
                                    settlement_days,
                                    face_amount,
                                    schedule,
                                    [r/100.0], # cupones todos iguales
                                    day_count,
                                    bussiness_convention)
    bond_helpers.append(helper)

rate_helpers = depo_helpers + bond_helpers

# GENERACION DE YIELD CURVE
yieldcurve = ql.PiecewiseLogCubicDiscount(calc_date, rate_helpers, day_count)

In [59]:
##########
# PROBANDO
##########


# print(list(schedule))

# coupons = [r/100. for _ in list(schedule)]

# new_helper = ql.FixedRateBondHelper(ql.QuoteHandle(ql.SimpleQuote(0)), # ?????
#                                     settlement_days,
#                                     face_amount,
#                                     schedule,
#                                     coupons,
#                                     day_count,
#                                     bussiness_convention)

# print ('PAYMENTS:', [cf.date() for cf in new_helper.bond().cashflows()])
# print ('COUPONS:', [round(cf.amount(),2) for cf in new_helper.bond().cashflows()])
# new_helper.bond().bondYield(72, day_count, compounding, ql.Semiannual)

In [64]:
# FLOATING RATE


In [9]:
# get spot rates
spots = []
tenors = []
for d in yieldcurve.dates():
    yrs = day_count.yearFraction(calc_date, d)
    compounding = ql.Compounded
    freq = ql.Semiannual
    zero_rate = yieldcurve.zeroRate(yrs, compounding, freq)
    eq_rate = zero_rate.equivalentRate(day_count,
                                       compounding,
                                       freq,
                                       calc_date, d).rate()
    # Create spot curve
    spots.append(round(eq_rate, 5))
    tenors.append(yrs)
    
print_curve(tenors, spots)

----------------------
Maturities	Curve
----------------------
0.0 		 0.0
0.5 		 0.052
1.0 		 0.054
1.5 		 0.058
2.0 		 0.06
2.5 		 0.063
3.0 		 0.066
3.5 		 0.068
4.0 		 0.069
4.5 		 0.071
5.0 		 0.072
5.5 		 0.073
6.0 		 0.073
6.5 		 0.074
7.0 		 0.075
7.5 		 0.075
8.0 		 0.077
8.5 		 0.078
9.0 		 0.078
9.5 		 0.079
10.0 		 0.081
----------------------
